In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset = pd.read_csv("./data/the-movies-dataset/ratings_small.csv")# 100k small data set
# dataset = pd.read_csv('/home/jeongchanwoo/바탕화면/ratings.csv')

In [3]:
dataset.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
len(dataset.userId.unique()), len(dataset.movieId.unique())

(671, 9066)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [6]:
### trainform data type int to object ( userId, movieId)
dataset.userId = dataset.userId.astype('category').cat.codes.values
dataset.movieId = dataset.movieId.astype('category').cat.codes.values

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int16
movieId      100004 non-null int16
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int16(2), int64(1)
memory usage: 1.9 MB


In [8]:
dataset.head()

,userId,movieId,rating,timestamp
0,0,30,2.5,1260759144
1,0,833,3.0,1260759179
2,0,859,3.0,1260759182
3,0,906,2.0,1260759185
4,0,931,4.0,1260759205


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2)

In [11]:
import keras
n_latent_factors_user = 8
n_latent_factors_movie = 10
n_latent_factors_mf = 3
n_users, n_movies = len(dataset.userId.unique()), len(dataset.movieId.unique())

movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding_mlp = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding-MLP')(movie_input)
movie_vec_mlp = keras.layers.Flatten(name='FlattenMovies-MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(n_movies + 1, n_latent_factors_mf, name='Movie-Embedding-MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name='FlattenMovies-MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.2)(movie_vec_mf)


user_input = keras.layers.Input(shape=[1],name='User')
user_vec_mlp = keras.layers.Flatten(name='FlattenUsers-MLP')(keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding-MLP')(user_input))
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_vec_mf = keras.layers.Flatten(name='FlattenUsers-MF')(keras.layers.Embedding(n_users + 1, n_latent_factors_mf,name='User-Embedding-MF')(user_input))
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)


concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp],name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = keras.layers.BatchNormalization(name='Batch')(dense)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name='Batch-2')(dense_2)


dropout_2 = keras.layers.Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)

pred_mf = keras.layers.multiply([movie_vec_mf, user_vec_mf], name='Dot')


pred_mlp = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)

combine_mlp_mf = keras.layers.concatenate([pred_mf, pred_mlp],name='Concat-MF-MLP')
result_combine = keras.layers.Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = keras.layers.Dense(100,name='FullyConnected-4')(result_combine)


result = keras.layers.Dense(1,name='Prediction')(deep_combine)


model = keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr =0.01)
model.compile(optimizer='adam',loss= 'mean_absolute_error')

Using TensorFlow backend.


In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding-MLP (Embedding) (None, 1, 10)        90670       Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding-MLP (Embedding)  (None, 1, 8)         5376        User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [13]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.0001, patience=7, verbose=1, mode='min') 
### 0.00001 이상의 변화가 없을 때 10회만큼 epoch 더 부여 후 중지

In [14]:
history = model.fit([train.userId, train.movieId], train.rating, epochs=10, verbose=1, validation_split=0.1,callbacks=[early_stopping])

Train on 72002 samples, validate on 8001 samples
Epoch 1/10
72002/72002 [==============================] - 7s 91us/step - loss: 0.8300 - val_loss: 0.7650
Epoch 2/10
72002/72002 [==============================] - 6s 81us/step - loss: 0.7313 - val_loss: 0.7032
Epoch 3/10
72002/72002 [==============================] - 6s 83us/step - loss: 0.6910 - val_loss: 0.7032
Epoch 4/10
72002/72002 [==============================] - 6s 82us/step - loss: 0.6658 - val_loss: 0.6877
Epoch 5/10
72002/72002 [==============================] - 6s 81us/step - loss: 0.6507 - val_loss: 0.7085
Epoch 6/10
72002/72002 [==============================] - 6s 81us/step - loss: 0.6395 - val_loss: 0.6967
Epoch 7/10
72002/72002 [==============================] - 6s 81us/step - loss: 0.6298 - val_loss: 0.6828
Epoch 8/10
72002/72002 [==============================] - 6s 81us/step - loss: 0.6234 - val_loss: 0.6825
Epoch 9/10
72002/72002 [==============================] - 6s 82us/step - loss: 0.6175 - val_loss: 0.6825
Epoch 

In [ ]:
model.save_weights()

In [23]:
y_true = test.rating

In [25]:
from sklearn.metrics import mean_absolute_error
y_hat_2 = np.round(model.predict([test.userId, test.movieId]),0)
print(mean_absolute_error(y_true, y_hat_2))

print(mean_absolute_error(y_true, model.predict([test.userId, test.movieId])))

0.6972651367431628
0.7055452523020039


In [34]:
len(test.userId.unique())

669

In [37]:
len(test.movieId.unique())

4845

In [38]:
test.shape

(20001, 4)

In [32]:
y_hat_2.shape

(20001, 1)

## Model creation

In [9]:
import tensorflow as tf

In [10]:
n_latent_factor_user = 8
n_latent_factor_moive = 10
n_latent_factor_mf = 3
n_users, n_movies = len(dataset.userId.unique()), len(dataset.movieId.unique())

In [11]:
dataset.movieId.unique()

array([  108,   145,   843, ..., 40896, 42982, 43046])

In [12]:
# movie_feature_column  = tf.feature_column.categorical_column_with_vocabulary_list(key='terms', vocabulary_list=dataset.movieId.unique())
from tensorflow import keras

In [13]:
movie_input = keras.layers.Input(shape = [1], name = 'Movie')
movie_embedding_mlp = keras.layers.Embedding(n_movies +1 , n_latent_factor_moive, name = 'Movie_embedding_MLP')(movie_input)
movie_vec_mlp = keras.layers.Flatten(name = 'FlattenMovies_MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(n_movies+1 , n_latent_factor_mf, name = 'Movie_embedding_MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name='FlattenMovies_MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.2)(movie_vec_mf)

user_input = keras.layers.Input(shape=[1],name='User')
user_embedding_mlp = keras.layers.Embedding(n_users +1 , n_latent_factor_user, name = 'Users_embedding_MLP')(user_input)
user_vec_mlp = keras.layers.Flatten(name = 'FlattenUser_MLP')(user_embedding_mlp)
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_embedding_mf = keras.layers.Embedding(n_users +1 , n_latent_factor_mf, name = 'Users_embedding_MF')(user_input)
user_vec_mf = keras.layers.Flatten(name = 'FlattenUser_MF')(user_embedding_mf)
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)

In [14]:
(movie_embedding_mf, movie_embedding_mlp, user_embedding_mf, user_embedding_mlp)

(<tf.Tensor 'Movie_embedding_MF/GatherV2:0' shape=(?, 1, 3) dtype=float32>,
 <tf.Tensor 'Movie_embedding_MLP/GatherV2:0' shape=(?, 1, 10) dtype=float32>,
 <tf.Tensor 'Users_embedding_MF/GatherV2:0' shape=(?, 1, 3) dtype=float32>,
 <tf.Tensor 'Users_embedding_MLP/GatherV2:0' shape=(?, 1, 8) dtype=float32>)

In [15]:
concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp])
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense_1 = keras.layers.Dense(200, name ='FullyConnected_1')(concat_dropout)
dense_batch_1 = keras.layers.BatchNormalization(name='Batch')(dense_1)
dropout_1 = keras.layers.Dropout(0.2, name ='Dropout_1')(dense_batch_1)
dense_2 = keras.layers.Dense(100, name = 'FullyConnected_2')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name = 'Batch_2')(dense_2)

dropout_2 = keras.layers.Dropout(0.2, name='Dropout_2')(dense_batch_2)
dense_3 = keras.layers.Dense(50, name = 'FullyConnected_3')(dropout_2)
dense_4 = keras.layers.Dense(20, name = 'FullyConnected_4', activation='relu')(dense_3)

In [16]:
(movie_vec_mlp.shape , user_vec_mlp.shape)

(TensorShape([Dimension(None), Dimension(10)]),
 TensorShape([Dimension(None), Dimension(8)]))

In [17]:
concat.shape

TensorShape([Dimension(None), Dimension(18)])

In [18]:
(user_vec_mf.shape, movie_vec_mf.shape)

(TensorShape([Dimension(None), Dimension(3)]),
 TensorShape([Dimension(None), Dimension(3)]))

In [19]:
movie_vec_mf

<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 3) dtype=float32>

In [20]:
user_vec_mf

<tf.Tensor 'dropout_3/cond/Merge:0' shape=(?, 3) dtype=float32>

In [21]:
# movie_vec_mf = keras.backend.transpose(movie_vec_mf) # movie_vec_mf shape=(?, 3) to (3,?)

In [22]:
movie_vec_mf

<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 3) dtype=float32>

In [23]:
pred_mf = keras.layers.multiply([movie_vec_mf, user_vec_mf], name = 'Elemet_wise_product')
pred_mlp = keras.layers.Dense(1, activation='relu', name = 'Activation_pred')(dense_4)

In [24]:
(pred_mf, pred_mlp)

(<tf.Tensor 'Elemet_wise_product/mul:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'Activation_pred/Relu:0' shape=(?, 1) dtype=float32>)

In [25]:
combine_mlp_mf = keras.layers.concatenate([pred_mf,pred_mlp],name = 'Concat_MF_MLP')
result_combine = keras.layers.Dense(100, name='Combine_MF_MLP')(combine_mlp_mf)

deep_combine = keras.layers.Dense(100, name = 'FullyConnected_5')(result_combine)
result = keras.layers.Dense(1, name='Prediction')(deep_combine)

model = keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr = 0.01)
model.compile(optimizer = 'adam', loss = 'mean_absolute_error',metrics=['mae'])

In [26]:
from IPython.display import SVG
# from tensorflow.keras.utils import plot_model

In [27]:
# SVG(plot_model(model, show_shapes=False,to_file='test_model', show_layer_names=True, rankdir='TB')) 
# SVG(plot_model(model, show_shapes=False, show_layer_names=True, rankdir='TB').create(prog='dot',format='svg'))

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie_embedding_MLP (Embedding) (None, 1, 10)        451160      Movie[0][0]                      
__________________________________________________________________________________________________
Users_embedding_MLP (Embedding) (None, 1, 8)         2167176     User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [30]:
# cv = KFold(n_splits=10, shuffle=True, random_state=0)
# cross_val_score(model, 
#                 dataset.userId, dataset.movieId, scoring="neg_mean_absolute_error", cv=cv)


In [31]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2)

In [33]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.0001, patience=7, verbose=1, mode='min') 
### 0.00001 이상의 변화가 없을 때 10회만큼 epoch 더 부여 후 중지

In [15]:
model_epoch_train = model.fit([train.userId, train.movieId], train.rating, epochs=100, verbose=1, 
                           validation_split=0.1, callbacks = [early_stopping])

Train on 72002 samples, validate on 8001 samples
Epoch 1/100
72002/72002 [==============================] - 6s 81us/step - loss: 0.6085 - val_loss: 0.6853
Epoch 2/100
72002/72002 [==============================] - 6s 81us/step - loss: 0.6063 - val_loss: 0.6810
Epoch 3/100
72002/72002 [==============================] - 6s 81us/step - loss: 0.6034 - val_loss: 0.6820
Epoch 4/100
72002/72002 [==============================] - 6s 80us/step - loss: 0.5984 - val_loss: 0.6877
Epoch 5/100
72002/72002 [==============================] - 6s 81us/step - loss: 0.5977 - val_loss: 0.6759
Epoch 6/100
72002/72002 [==============================] - 6s 83us/step - loss: 0.5961 - val_loss: 0.6809
Epoch 7/100
72002/72002 [==============================] - 6s 84us/step - loss: 0.5922 - val_loss: 0.6827
Epoch 8/100
72002/72002 [==============================] - 6s 82us/step - loss: 0.5911 - val_loss: 0.6797
Epoch 9/100
72002/72002 [==============================] - 6s 81us/step - loss: 0.5894 - val_loss: 0.67

In [ ]:
from sklearn.metrics import mean_absolute_error


In [ ]:
y_predict = np.round(model.predict([test.userId, test.movieId]), 0)

In [ ]:
print(mean_absolute_error(test.rating, y_predict))
print(mean_absolute_error(test.rating,model.predict([test.userId, test.movieId])))

In [ ]:
import h5py

In [ ]:
# h5py.run_tests()

In [17]:
keras.models.save_model(model=model_epoch_train, filepath='./data/model/neural_MF_1_테스팅')
# tf.keras.models.save_model(model=model, filepath='./data/model/neural_MF_1_big_file.h5')

AttributeError: 'History' object has no attribute 'get_config'

In [50]:
model = keras.models.load_model('./data/model/neural_MF_1.h5',)

In [54]:
model

In [51]:
# y_predict_big = np.round(model.predict([dataset.userId,dataset.movieId]), 0)

# print(mean_absolute_error(dataset.rating, y_predict_big))
# print(mean_absolute_error(dataset.rating, model.predict([dataset.userId,dataset.movieId])))

InvalidArgumentError: indices[12,0] = 10177 is not in [0, 9067)
	 [[Node: Movie_embedding_MLP_1/GatherV2 = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Movie_embedding_MLP_1/embeddings/read, Movie_embedding_MF_1/Cast, Movie_embedding_MF_1/GatherV2/axis)]]

Caused by op 'Movie_embedding_MLP_1/GatherV2', defined at:
  File "/home/jeongchanwoo/miniconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jeongchanwoo/miniconda3/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jeongchanwoo/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jeongchanwoo/miniconda3/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jeongchanwoo